In [211]:
from bs4 import BeautifulSoup,element
import pandas as pd
from glob import glob

In [212]:
df_deputy = pd.read_csv("data/nosdeputes.fr_deputes_en_mandat_2023-08-02.csv",sep=";")
id2slug = dict(df_deputy["id_an slug".split()].values)

In [216]:
from datetime import date,datetime
from dataclasses import dataclass
from typing import Any

@dataclass
class Intervention:
    section:str
    subsection:str
    text:str
    timestamp: float
    orateur_nom:str
    orateur_slug:str
    
    def tolist(self) -> list[Any]:
        return [self.section,self.subsection,self.text,self.timestamp,self.orateur_nom,self.orateur_slug]

class SeanceParser:
    def __init__(self,xml_string:str) ->None:
        self.__data:BeautifulSoup = BeautifulSoup(xml_string,features="xml")

    
    def parse_metadata(self) -> None:
        meta : element.Tag = self.__data.find("metadonnees")

        def get_field(field:str):
            return meta.select_one(field).text

        string_date:str = get_field("dateSeance")
        (year,month,day) = int(string_date[:4]),int(string_date[4:6]),int(string_date[6:8])
        self.date_seance:date = date(year,month,day)

        self.num_seance = int(get_field("numSeance"))

        self.legislature:int = int(get_field("legislature"))
        self.presidence:str = get_field("presidentSeance")
        self.presidence_id: str = meta.select_one("presidentSeance").attrs["id_syceron"]
        

    def parse_intervention(self) -> None:
        self.interventions: list[Intervention] = []
        current_section:str = ""
        current_subsection:str = ""

        for point in self.__data.find_all('point'):
            level:int = int(point.attrs["nivpoint"])
            title:str = point.select_one("texte").text
            if level == 1:
                current_section = title
            elif level == 2:
                current_subsection = title
            for paragraphe in point.find_all("paragraphe"):
                orateur = paragraphe.select_one("orateur")
                inter = paragraphe.select_one("texte")
                if orateur and inter:
                    identifiant:int = int(orateur.id.text)
                    slug = "UNKNOWN"
                    if identifiant in id2slug:
                        slug = id2slug[identifiant]
                    elif identifiant != 0:
                        slug = "GOV"
                    self.interventions.append(Intervention(*[current_section,current_subsection,inter.text,
                                                             eval(inter.attrs["stime"]) if "stime" in inter.attrs else 0,
                                                             orateur.nom.text,slug]))

    def to_dataframe(self)->pd.DataFrame:
        assert hasattr(self,"interventions")
        assert hasattr(self,"legislature")
        assert hasattr(self,"presidence")
        assert hasattr(self,"presidence_id")
        assert hasattr(self,"num_seance")
        assert hasattr(self,"date_seance")

        data = [[self.legislature,self.date_seance,self.num_seance,self.presidence,self.presidence_id,*intervention.tolist()]for intervention in self.interventions]
        return pd.DataFrame(data,columns="legislature date numero_seance nom_president_e id_president_e section subsection full_text timestamp speaker_name slug".split())

        

        

In [217]:
from tqdm.notebook import tqdm
dataframes = []
for filename in tqdm(glob("data/raw_data/compteRendu/*.xml")):
    xml_str = open(filename,encoding="utf-8").read()
    s = SeanceParser(xml_str)
    s.parse_metadata()
    s.parse_intervention()
    dataframes.append(s.to_dataframe())

df = pd.concat(dataframes)

  0%|          | 0/357 [00:00<?, ?it/s]

In [218]:
df

,legislature,date,numero_seance,nom_president_e,id_president_e,section,subsection,full_text,timestamp,speaker_name,slug
0,16,2021-02-01,144,Présidence de M. Richard Ferrand,2384838,Respect des principes de la République,"\nDiscussion, après engagement de la procédure...",L’ordre du jour appelle la discussion du proje...,481.57,M. le président,GOV
1,16,2021-02-01,144,Présidence de M. Richard Ferrand,2384838,Respect des principes de la République,Présentation,La parole est à M. le ministre de l’intérieur.,491.77,M. le président,GOV
2,16,2021-02-01,144,Présidence de M. Richard Ferrand,2384838,Respect des principes de la République,Présentation,"Monsieur le président, mesdames, messieurs les...",531.62,M. Gérald Darmanin,GOV
3,16,2021-02-01,144,Présidence de M. Richard Ferrand,2384838,Respect des principes de la République,Présentation,Si ! C’est ce que vous faites.,600.27,M. Jean-Luc Mélenchon,GOV
4,16,2021-02-01,144,Présidence de M. Richard Ferrand,2384838,Respect des principes de la République,Présentation,"…mais un texte pour tout le monde, qui aura de...",674.55,M. Gérald Darmanin,GOV
...,...,...,...,...,...,...,...,...,...,...,...
338,16,2023-06-29,293,Présidence de Mme Caroline Fiat,3180658,Restitution des biens culturels spoliés entre ...,Explications de vote,La parole est à Mme Caroline Parmentier.,5089.16,Mme la présidente,caroline-fiat
339,16,2023-06-29,293,Présidence de Mme Caroline Fiat,3180658,Restitution des biens culturels spoliés entre ...,Explications de vote,Ce projet de loi marque une nouvelle étape de ...,5098.14,Mme Caroline Parmentier (RN),caroline-parmentier
340,16,2023-06-29,293,Présidence de Mme Caroline Fiat,3180658,Restitution des biens culturels spoliés entre ...,Vote sur l’ensemble,Je mets aux voix l’ensemble du projet de loi.,5142.17,Mme la présidente,caroline-fiat
341,16,2023-06-29,293,Présidence de Mme Caroline Fiat,3180658,Restitution des biens culturels spoliés entre ...,Vote sur l’ensemble,Voici le résultat du scrutin : Nombre d...,5158.80,Mme la présidente,caroline-fiat


In [97]:
for child in tree.getroot():
    print(child)

<Element '{http://schemas.assemblee-nationale.fr/referentiel}uid' at 0x0000021D208EF860>
<Element '{http://schemas.assemblee-nationale.fr/referentiel}seanceRef' at 0x0000021D208EF900>
<Element '{http://schemas.assemblee-nationale.fr/referentiel}sessionRef' at 0x0000021D208EF9A0>
<Element '{http://schemas.assemblee-nationale.fr/referentiel}metadonnees' at 0x0000021D208EF9F0>
<Element '{http://schemas.assemblee-nationale.fr/referentiel}contenu' at 0x0000021D208F5EF0>


In [92]:
root.items()

[]

In [77]:
dataSeance["compteRendu"]["contenu"]["point"][2]

{'@nivpoint': '2',
 '@valeur_ptsodj': '1',
 '@ordinal_prise': '1',
 '@id_preparation': '1791380',
 '@ordre_absolu_seance': '8',
 '@code_grammaire': 'PRESENTATION_1_0',
 '@code_style': 'Sous-tit_p_cap',
 '@code_parole': '',
 '@sommaire': '1',
 '@id_syceron': '2384844',
 '@valeur': '',
 'orateurs': None,
 'texte': 'Présentation',
 'interExtraction': [{'@nom_orateur': "M.\xa0Gérald Darmanin, quali  ministre de l'intérieur",
   '@id_acteur': 'PA607846',
   '@id_nomination_oe': 'PM773469',
   '@id_nomination_op': '-1',
   '@id_mandat': '-1',
   'paragraphe': [{'@valeur_ptsodj': '1',
     '@ordinal_prise': '1',
     '@id_preparation': '1791381',
     '@ordre_absolu_seance': '9',
     '@id_acteur': 'PA606171',
     '@id_mandat': 'PM722632',
     '@id_nomination_oe': '-1',
     '@id_nomination_op': 'PM758870',
     '@code_grammaire': 'PAROLE_GENERIQUE',
     '@code_style': 'NORMAL',
     '@code_parole': 'PAROLE_1_1',
     '@sommaire': '0',
     '@id_syceron': '2384845',
     '@valeur': '',
   

[('__str__', <class 'method-wrapper'>),
 ('__subclasshook__', <class 'builtin_function_or_method'>),
 ('__weakref__', <class 'NoneType'>),
 ('date_seance', <class 'datetime.date'>),
 ('get_text', <class 'method'>),
 ('legislature', <class 'int'>),
 ('num_seance', <class 'int'>),
 ('parse_intervention', <class 'method'>),
 ('parse_metadata', <class 'method'>),
 ('presidence', <class 'str'>),
 ('presidence_id', <class 'str'>)]


TypeError: string indices must be integers